In [9]:
import pandas as pd

# Excel 파일 경로 (Google Drive에 업로드한 파일의 경로)
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/Colab Notebooks/june/Sample_Test.xlsx'
# 엑셀 파일 읽기
data = pd.read_excel(file_path)
print(data.head())


Mounted at /content/drive
   No.       JN  Project  FI Brand  \
0    1  JN40022  PC Body  WY   g&h   
1    2  JN40022  PC Body  WY   g&h   
2    3  JN40022  PC Body  WY   g&h   
3    4  JN40022  PC Body  WY   g&h   
4    5  JN40022  PC Body  WY   g&h   

                                        Product   SKU   Launch  Lab  \
0                             Nourish Foot Cream    7  2023-Q1  CRL   
1                             Nourish Hand Cream   10  2023-Q1  CRL   
2                               Nourish Lip Balm    6  2023-Q1  CRL   
3  Protect UV Sunscreen SPF 50+ PA ++++ (GLOBAL)    9  2023-Q2  CRL   
4                    Nourish Foot Cream (Taiwan)    1  2023-Q1  CRL   

          Study_Code Test_Start   Test_End  
0  CRLNJ2021-0703-11 2021-09-03 2021-10-15  
1  CRLNJ2021-0703-11 2021-09-03 2021-10-15  
2  CRLNJ2021-0703-11 2021-09-03 2021-10-15  
3  CRLNJ2021-0703-11 2021-09-03 2021-10-15  
4  CRLNJ2021-0703-11 2021-09-03 2021-10-15  


In [10]:
# Test_Start와 Test_End 열을 datetime 형식으로 변환
data['Test_Start'] = pd.to_datetime(data['Test_Start'])
data['Test_End'] = pd.to_datetime(data['Test_End'])

# 데이터 확인
print(data[['Test_Start', 'Test_End']].head())


  Test_Start   Test_End
0 2021-09-03 2021-10-15
1 2021-09-03 2021-10-15
2 2021-09-03 2021-10-15
3 2021-09-03 2021-10-15
4 2021-09-03 2021-10-15


In [11]:
# 열 이름을 소문자로 변환하고 공백을 언더스코어로 대체
data.columns = data.columns.str.lower().str.replace(' ', '_').str.replace('[^a-z0-9_]', '')

# 데이터 확인
print(data.columns)


Index(['no.', 'jn', 'project', 'fi', 'brand', 'product_', 'sku', 'launch',
       'lab', 'study_code', 'test_start', 'test_end'],
      dtype='object')


In [13]:
import pandas as pd
import sqlite3

# 엑셀 파일 읽기
# file_path = '/mnt/data/Modified_File.xlsx'
# data = pd.read_excel(file_path)

# SQLite 데이터베이스 연결 (파일 생성)
conn = sqlite3.connect('data.db')

# 데이터 저장 (데이터베이스에 테이블 생성 및 삽입)
data.to_sql('data_table', conn, if_exists='replace', index=False)

print("엑셀 데이터를 SQLite 데이터베이스로 변환 완료!")


엑셀 데이터를 SQLite 데이터베이스로 변환 완료!


In [31]:
from flask import Flask, request, render_template
import sqlite3

app = Flask(__name__)

# 데이터베이스 연결 함수
def query_database(query):
    conn = sqlite3.connect('data.db')
    cursor = conn.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    conn.close()
    return results

# 메인 페이지
@app.route('/')
def home():
    return '''
        <form action="/search" method="get">
            검색어: <input type="text" name="query">
            <input type="submit" value="검색">
        </form>
    '''

# 검색 기능 구현
@app.route('/search')
def search():
    query = request.args.get('query')
    if query:
        sql_query = f"SELECT * FROM data_table WHERE Product LIKE '%{query}%'"
        results = query_database(sql_query)
        return f"<h2>검색 결과:</h2> {results}"
    else:
        return "검색어를 입력하세요."

if __name__ == '__main__':
    app.run(debug=True,port=5001)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


# 칼럼 별 검색 추가

In [3]:
from flask import Flask, request, render_template
import sqlite3

app = Flask(__name__)

# 데이터베이스 연결 함수
def query_database(query, params=None):
    conn = sqlite3.connect('data.db')
    cursor = conn.cursor()
    cursor.execute(query, params or [])
    results = cursor.fetchall()
    conn.close()
    return results

# 메인 페이지
@app.route('/')
def home():
    return render_template('home.html')

# 검색 기능 구현
@app.route('/search')
def search():
    query = request.args.get('query')
    column = request.args.get('column')
    if query and column:
        # SQL Injection 방지를 위해 플레이스홀더 사용
        sql_query = f"SELECT * FROM data_table WHERE {column} LIKE ?"
        results = query_database(sql_query, (f"%{query}%",))
        return render_template('results.html', query=query, column=column, results=results)
    else:
        return "검색어와 칼럼을 모두 입력하세요."

if __name__ == '__main__':
    app.run(debug=True, port=5001)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/opt/anaconda3/envs/venv/lib/python3.8/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/envs/venv/lib/python3.8/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/opt/anaconda3/envs/venv/lib/python3.8/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/opt/anaconda3/envs/venv/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/opt/anaconda3/envs/venv/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 331, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/opt/anaconda3/envs/venv/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 253, in _bind_socket
    return

SystemExit: 1